In [1]:
import numpy as np

In [2]:
def Convolution(Spectrum):
    n = len(Spectrum)
    Cov_Spectrum = []
    for i in range(n-1):
        for j in range(i+1,n):
            if(Spectrum[j]-Spectrum[i]>0):
                Cov_Spectrum.append(Spectrum[j]-Spectrum[i])
    return Cov_Spectrum
            

In [3]:
def Quota(Peptide,mass):
    used = 0
    PeptideList = Peptide.split('-')
    for amino_acid in PeptideList:
        if(mass == int(amino_acid)):
            used += 1
    return Total[mass]-used

In [4]:
def SelectMost(Cov_Spectrum,M):
    Count = {}
    Select_Spectrum = []
    for ele in Cov_Spectrum:
        if ele in Count.keys():
            Count[ele] += 1
        else:
            Count[ele] = 1
    c = 0
    pre = -1
    for mass,count in sorted(Count.items(), key=lambda item: item[1],reverse=True):
        if(c<M or pre == count):
            if(mass>=57 and mass<=200):
                c +=1
                Select_Spectrum += [mass]*count
                pre= count
        
    return np.sort(Select_Spectrum)
        

In [5]:
def LinearSpectrum(Peptide):
    Peptides = Peptide.split('-')
    #Peptides = Peptide
    n = len(Peptides)
    PrefixMass = np.zeros(n+1)
    for i in range(0,n):
        PrefixMass[i+1] = PrefixMass[i] + int(Peptides[i])
    LinearSpectrum = [0]
    for i in range(n):
        for j in range(i+1,n+1):
            #add PrefixMass(j) − PrefixMass(i) to LinearSpectrum
            LinearSpectrum.append(PrefixMass[j]-PrefixMass[i])
            
    return np.sort(LinearSpectrum)

In [6]:
def Mass(Peptide):
    mass = 0
    PeptideList = Peptide.split('-')
    #PeptideList = Peptide
    for amino_acid in PeptideList:
        mass += int(amino_acid)
    return mass

In [34]:
def Expand(CandidatePeptides):
    if (not CandidatePeptides):
        #return [[k] for k in set(AminoAcidMass.values())]
        #return set(str(k) for k in set(AminoAcidMass.values()))
        return [str(k) for k in Alphat]
    NewCandidatePeptides = []
    for Peptide in CandidatePeptides:
        for mass in Alphat:
        #for mass in set(AminoAcidMass.values()):
            if(Quota(Peptide,mass)>0):
            #NewPeptide = Peptide + [mass]
                NewPeptide = Peptide + '-'+str(mass)
            #if not CheckRepeatPeptide(NewPeptide,NewCandidatePeptides):
                NewCandidatePeptides.append(NewPeptide)
    return NewCandidatePeptides

In [8]:
def Score(Peptide,Experimental):
    if(not Peptide):
        return 0
    Theoretical = LinearSpectrum(Peptide)
    Total = {}
    for mass in Theoretical:
        if mass in Total.keys():
            Total[mass] += 1
        else:
            Total[mass] = 1
    score = 0
    for mass in Experimental:
        if(mass in Total.keys() and Total[mass]>0):
            score += 1
            Total[mass] -= 1
    return score

In [9]:
def Trim(Leaderboard, Spectrum, N):
    n = len(Leaderboard)
    LinearScores = {}
    for Peptide in Leaderboard:
        LinearScores[Peptide] =  Score(Peptide, Spectrum)
    
    count = 0
    NewLeaderBoard = []
    preScore = -1
    for Peptide,score in sorted(LinearScores.items(), key=lambda item: item[1],reverse=True):
        #Peptide = Peptide.strip('][').split(', ') 
        #Peptide = [int(x) for x in Peptide]
        #k = len(Peptide)
        if(preScore == score or count<N):
            NewLeaderBoard.append(Peptide)
        else:
            break
        count += 1
        preScore = score
    return NewLeaderBoard

In [181]:
def LeaderboardCyclopeptideSequencing(Sel_Spectrum,Spectrum, N):
    LeaderPeptide = ''
    Leaderboard  = []
    while True:
        Leaderboard = Expand(Leaderboard)
        Leaderboard = Trim(Leaderboard, Spectrum, N)
        DiscardBoard = []
        for Peptide in Leaderboard:
                #if Mass(LinearSpectrum(Peptide)) == parentMass:
            if Mass(Peptide) ==  Spectrum[-1]:
                if Score(Peptide, Sel_Spectrum) > Score(LeaderPeptide, Sel_Spectrum):
                    LeaderPeptide = Peptide
                DiscardBoard.append(Peptide)
            elif Mass(Peptide) > Spectrum[-1]:
                DiscardBoard.append(Peptide)
        Leaderboard = [x for x in Leaderboard if x not in DiscardBoard]
        if not Leaderboard:
            return LeaderPeptide

In [182]:
N = 20
M = 60
Spectrum = '0 57 57 71 99 129 137 170 186 194 208 228 265 285 299 307 323 356 364 394 422 493'
Spectrum = np.array(Spectrum.split(' ')).astype(int)
Spectrum = np.sort(Spectrum)

In [183]:
Cov_Spectrum = Convolution(Spectrum)

In [184]:
Sel_Spectrum = SelectMost(Cov_Spectrum,M)
Alphat = set(Sel_Spectrum)
Sel_Spectrum = np.sort(Sel_Spectrum)
Total = {}
for mass in Cov_Spectrum:
    if mass in Total.keys():
        Total[mass] += 1
    else:
        Total[mass] = 1
LeaderPeptide = ''

In [185]:
LeaderboardCyclopeptideSequencing(Sel_Spectrum, Spectrum,N)

'71-99-129-57-79-58'

In [153]:
Leaderboard = []

In [162]:
Leaderboard = Expand(Leaderboard)
Leaderboard = Trim(Leaderboard,Spectrum, N)
DiscardBoard = []
for Peptide in Leaderboard:
    if Mass(Peptide) ==  Spectrum[-1]:
        Score1 = Score(Peptide, Spectrum)
        Score2 = Score(LeaderPeptide, Spectrum)
        print("LeaderPeptide:%s\tScore1: %d\tScore2: %d"%(LeaderPeptide,Score1,Score2))
        if (Score(Peptide, Spectrum) > Score(LeaderPeptide, Spectrum)):
            print(LeaderPeptide,Peptide)
            LeaderPeptide = Peptide
        DiscardBoard.append(Peptide)
    elif Mass(Peptide) > Spectrum[-1]-57:
            DiscardBoard.append(Peptide)
if len(DiscardBoard) == len(Leaderboard):
    print("Stop")

print("LeaderBoard: %s"%(str(Leaderboard)))
print("DiscardBoard: %s"%(str(DiscardBoard)))
Leaderboard = [x for x in Leaderboard if x not in DiscardBoard] 
print(Leaderboard)

Stop
LeaderBoard: ['57-129-99-71-66-57-71', '57-129-99-71-66-57-200', '57-137-71-99-58-57-71', '57-137-71-99-58-57-129', '57-137-71-99-58-57-109', '57-129-99-71-66-57-129', '57-129-99-71-66-57-151', '57-129-99-71-66-57-170', '57-129-99-71-66-57-113', '57-137-71-99-58-57-186', '57-137-71-99-58-57-72', '57-129-99-71-66-57-128', '57-129-99-71-66-57-136', '57-129-99-71-66-57-137', '57-129-99-71-66-57-148', '57-129-99-71-66-57-153', '57-129-99-71-66-57-156', '57-129-99-71-66-57-157', '57-129-99-71-66-57-162', '57-129-99-71-66-57-166', '57-129-99-71-66-57-171', '57-129-99-71-66-57-178', '57-129-99-71-66-57-57', '57-129-99-71-66-57-58', '57-129-99-71-66-57-186', '57-129-99-71-66-57-65', '57-129-99-71-66-57-66', '57-129-99-71-66-57-194', '57-129-99-71-66-57-72', '57-129-99-71-66-57-77', '57-129-99-71-66-57-79', '57-129-99-71-66-57-80', '57-129-99-71-66-57-87', '57-129-99-71-66-57-91', '57-129-99-71-66-57-95', '57-129-99-71-66-57-99', '57-129-99-71-66-57-105', '57-129-99-71-66-57-109', '57-129-

In [190]:
N=20
M=373
line = '853 113 585 796 924 502 423 1210 342 186 761 391 593 1412 1152 1396 260 129 1381 229 242 356 990 1047 57 748 1176 730 990 1038 1119 294 339 114 696 1251 1267 617 567 357 471 163 1266 1281 0 536 1395 454 1104 1362 1039 892 1509 1086 129 649 1095 713 258 777 1394 753 299 599 648 876 414 1249 813 242 859 1305 552 1284 861 650 1249 261 520 470 519 957 1233 405 260 861 762 810 1248 891 916 1346 390 981 147 1323 390 732 618 1380 1038 756 989 225 633 910 204 1452 243 1119 860 1395 129 57 503 1267 1153 276 462 228 1215 114 1170 357 973 388 519 699 131 128 1120 648 1452 1055 632 333 1380 528 747 389 656 97 1167 779 1380 1280 942 115 1121 1152 1007 990 1006 1118 519 877 1378 471'
Spectrum = np.array(line.split(' ')).astype(int)

In [195]:
N = 0
Spectrum = []
M=0
file = open('dataset_369297_7.txt', 'r') 
for i, line in enumerate(file):
    line=line.rstrip('\n')
    if(i==0):
        N = int(line)
    elif(i==1):
        M = int(line)
    else:
        Spectrum = np.array(line.split(' ')).astype(int)


In [196]:
Spectrum = np.sort(Spectrum)
Cov_Spectrum = Convolution(Spectrum)
Sel_Spectrum = SelectMost(Cov_Spectrum,M)
Alphat = set(Sel_Spectrum)
Sel_Spectrum = np.sort(Sel_Spectrum)
Total = {}
for mass in Cov_Spectrum:
    if mass in Total.keys():
        Total[mass] += 1
    else:
        Total[mass] = 1
LeaderPeptide = ''

In [197]:
LeaderboardCyclopeptideSequencing(Sel_Spectrum, Spectrum,N)

'71-99-113-147-156-103-71-57-57-113-128-137'

In [193]:
Score('57-57-129-113-115-114-128-97-163-131-129-129-147',Spectrum)

92

In [194]:
Score('113-115-114-128-97-163-131-129-129-147-57-57-129',Spectrum)

92